In [ ]:
import os
import pandas as pd
from ons import get_sheet_names, detect_id_row, get_headers, clean_measure_column, clean_measure_type_column, add_group, load_data, create_date_column, melt_table

In [ ]:
unem01_file = '../../data/raw/unem01-latest.xls'

In [ ]:
sheets = get_sheet_names(unem01_file)
sheets

In [ ]:
id_row = detect_id_row(unem01_file)

In [ ]:
headers = pd.concat(
    [
        get_headers(unem01_file, s, id_row)
        .pipe(clean_measure_column)
        .pipe(clean_measure_type_column)
        .pipe(add_group, s)
        for s in sheets[1:]
    ]).sort_index()
headers.loc[headers.measure.str.match('%'), ['measure_type']] = 'rate %'
headers

In [ ]:
headers_file = '../../data/metadata/labour-market/unem01-codes.csv'
os.makedirs(os.path.dirname(headers_file), exist_ok=True)
headers.to_csv(headers_file)

In [ ]:
data = pd.concat([
  load_data(unem01_file, s, id_row).pipe(create_date_column).pipe(melt_table).merge(headers, left_on='variable_name', right_index=True)
  for s
  in sheets[1:]
])

In [ ]:
out_file = '../../data/processed/labour-market/unem01.csv'
os.makedirs(os.path.dirname(out_file), exist_ok=True)
data.to_csv(out_file, index=False)